In [3]:
import pandas as pd
import os

In [ ]:
sheet_no_mod = ['Par_TradeRoute',
             'Par_TradeCosts',
             'Par_TradeCapacity',
             'Par_TradeCapacityGrowthCosts',
             'Par_GrowthRateTradeCapacity',
             'Par_RegionalAnnualEmissionLimit',
             'Par_EmissionsPenalty',
             'Par_SpecifiedAnnualDemand',
             'Par_ReserveMargin',
             'Par_ResidualCapacity',
             'Par_AvailabilityFactor',
             'Par_TotalAnnualMaxActivity',
             'Par_TotalAnnualMaxCapacity',
             'Par_ModelPeriodActivityMaxLimit',
             'Par_RegionalCCSLimit',
             'Par_RegionalBaseYearProduction',
             'Par_ModalSplitByFuel',
             'Sets',
             'Par_DaySplit',
             'Par_YearSplit',
             'Par_TagTechnologyToSector',
             'Par_CapacityToActivityUnit',
             'Par_Conversionls',
             'Par_Conversionld',
             'Par_Conversionlh',
             'Par_AnnualExogenousEmission',
             'Par_AnnualEmissionLimit',
             'Par_AnnualSectoralEmissionLimit',
             'Par_EmissionActivityRatio',
             'Par_EmissionPenaltyTagTech',
             'Par_EmissionContentPerFuel',
             'Par_ReserveMarginTagFuel',
             'Par_ReserveMarginTagTechnology',
             'Par_InputActivityRatio',
             'Par_OutputActivityRatio',
             'Par_CapitalCost',
             'Par_VariableCost',
             'Par_FixedCost',
             'Par_CapacityFactor',
             'Par_OperationalLife',
             'Par_TotalAnnualMinActivity',
             'Par_TotalAnnualMinCapacity',
             'Par_TagTechnologyToModalType',
             'Par_BaseYearProduction',
             'Par_TechnologyToStorage',
             'Par_TechnologyFromStorage',
             'Par_StorageLevelStart',
             'Par_StorageMaxChargeRate',
             'Par_StorageMaxDischargeRate',
             'Par_MinStorageCharge',
             'Par_OperationalLifeStorage',
             'Par_CapitalCostStorage',
             'Par_ResidualStorageCapacity']


In [ ]:
class DataRaw:

    def __init__(self, filepath, outputpath):
        self.filepath = filepath
        self.outputpath = outputpath
        self.name = None
        self.dic_sheets = {}
        self.df = pd.DataFrame
        self.process_excel()
        
    def process_excel(self):

            """
            Iterate through Excel sheets and make dictionary
            """
            for sheet in sheet_no_mod:
                self.name = sheet
                print(self.name)
               
                df_first = pd.read_excel(self.filepath,
                                        sheet_name=sheet,
                                        engine='openpyxl', header=None)

                r = 5
                df_final = df_first.iloc[:r] #shweta
                df_final = df_final.apply(lambda x: x.apply(lambda y: str(y).replace('.0', '')))
                df_raw = df_first.iloc[r:]
                df_raw = self.remove_blank_rows(df_raw)
                df_raw = df_raw.apply(lambda x: x.apply(lambda y: str(y).replace('.', ',')))
                df = pd.concat([df_final, df_raw], axis=0)
                df.replace('nan', '', inplace=True) # remove 'nan' with nothing
                df.to_csv((self.outputpath + f'{self.name}.csv'), sep=';', index=False, header=False)

                
    def remove_blank_rows(self, df):
        """
        :param df pandas dataframe from excel sheet
        :return: dataframe with removed nan values
        """
        # remove blank rows
        try:
            df = df.loc[0:df[df.isnull().all(axis=1) == True].index.tolist()[0] - 1]
        except IndexError:
            pass

        return df

In [ ]:
FILENAME = "../../NO_disagg_GradualDevelopment_oE_v492.xlsx"

FILE_EXTENSION = "csv"

PATH_TO_CSV = "../../csvs/"

df = DataRaw(filepath=FILENAME,
                  outputpath = PATH_TO_CSV)


In [8]:
"""
Method to read the csv files from the folder one by one
"""
class Process_csv:
    def __init__(self, folderpath):
        self.folderpath = folderpath
        self.name = None
        self.df = pd.DataFrame
        #self.filter_region = filter_region
        self.read_csvs()
        
    def read_csvs(self): 
        # Loop through all the files in the folder
        for filename in os.listdir(self.folderpath):
            if filename.endswith('.csv'):  # Ensure the file is a CSV file
                file_path = os.path.join(self.folderpath, filename)
                if filename == 'Par_ResidualCapacity.csv':
                    # Read the CSV file and create a DataFrame
                    self.df = pd.read_csv(file_path,  sep=';', skiprows=4)
                    if self.df[(self.df.columns[0] == 'AT')]:
                        filtered_df = self.df
                    
                    #filtered_df = self.df[self.df.columns[0] == 'AL']
                    print(self.df.shape)
        return df

In [9]:
FOLDER_PATH = '/Users/shwetat/Projects/Ocean_grid/Havnett_repo/csvs/'
#filter_region = ['AT', 'SE']
Process_csv(folderpath=FOLDER_PATH)

KeyError: False

In [ ]:
# Functions to combine csvs into 1 xlsx file
def get_file_list(path_to_csvs, file_extension):
    """
    Calls function to get list of all csv files
    :para path_to_csv: path to csv folder
    :para file_extension: csv in this case
    :return list of csv file names
    """
    filenames = []
    for file in os.listdir(path_to_csvs):
        if file.endswith(file_extension):
            filenames.append(file)
    return filenames
    
def combine_xlsx(path_to_csvs, all_filenames, output_file):  
    """
    Calls function to combine csv files
    :para path_to_csv: path to csv folder
    :para all_filenames: list of csv files in the folder
    :para output_file: name and extension of output file
    :return list of csv file names
    """
    print(path_to_csvs)
    writer = pd.ExcelWriter(path_to_csvs + output_file) 
    for csvfilename in all_filenames: 
        print("Loading "+ csvfilename)
        df = pd.read_csv(path_to_csvs + csvfilename, sep=';', encoding='utf-8')
        # df_final =  df1.iloc[:4, :1]
        # df_t = df1[4:]
        # df_t = df_t.apply(lambda x: x.apply(lambda y: str(y).replace('.', ',')))
        # df = pd.concat([df_final, df_t], axis=0)
        for j in range(len(df.columns)):
            if j == 0:
                print('do nothing')
            else:
                old = df.columns[j]
                new = ''
                df = df.rename(columns = {old:new})
        df.to_excel(writer, sheet_name=csvfilename[:-4], index=False, header=True)
        
        print("done")
        #break
    writer.save()
    print("task completed")